In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import time
import sklearn
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

In [2]:
import pandas as pd

train_df = pd.read_csv("DiabetesTrain.csv", index_col=0)
test_df = pd.read_csv("DiabetesTest.csv", index_col=0)

In [3]:
x_train = train_df.drop('Diabetes', axis=1)
y_train = train_df['Diabetes']

# Implementation of perceptron 

In [4]:
x_biased =  pd.concat([pd.Series(1, index=x_train.index, name='intercept'), x_train], axis=1)

In [5]:
x_biased

,intercept,Age,Sex,HighChol,CholCheck,BMI,Smoker,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,GenHlth,MentHlth,PhysHlth,DiffWalk,Stroke,HighBP
1,1,10.0,1.0,1.0,1.0,33.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,30.0,0.0,0.0,1.0
2,1,10.0,1.0,0.0,1.0,21.0,0.0,1.0,1.0,0.0,0.0,0.0,5.0,0.0,30.0,1.0,0.0,1.0
3,1,4.0,0.0,0.0,1.0,32.0,0.0,0.0,1.0,1.0,1.0,0.0,3.0,0.0,7.0,0.0,0.0,0.0
4,1,11.0,1.0,1.0,1.0,35.0,1.0,0.0,1.0,0.0,1.0,0.0,4.0,0.0,10.0,1.0,0.0,1.0
5,1,10.0,0.0,1.0,1.0,27.0,0.0,0.0,1.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,1,9.0,1.0,1.0,1.0,44.0,1.0,0.0,1.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0
997,1,3.0,0.0,1.0,1.0,33.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
998,1,9.0,0.0,1.0,1.0,41.0,0.0,1.0,1.0,0.0,1.0,0.0,5.0,20.0,30.0,1.0,1.0,1.0
999,1,12.0,0.0,1.0,1.0,34.0,0.0,1.0,1.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0


In [68]:
def perceptron(x, y, b, w_init, eta, epoch):
    """
    Implements a binary single-example perceptron with margin
    Inputs:    x a feature matrix containing an example on each row [pandas DataFrame of shape n x d]
               y a vector with the class (either 1 or 0) of each example  [numpy array of size n]
               b a margin [float]
               w_init a vector with the initial weight values (intercept in w_init[0]) [numpy array of size d+1]
               eta a fixed learning rate [float]
               epoch the maximal number of iterations (1 epoch = 1 iteration
                       of the "repeat" loop in the lecture slides) [int]
    Output:    A weight vector [list or numpy array of size d+1] (intercept in w[0])
    """
    # for bias; as we will multiply with the weights we need a first column of just "1". 
    x_biased = pd.concat([pd.Series(1, index=x.index, name='intercept'), x], axis=1)
    x_biased = x_biased.to_numpy()
    
    y = y.to_numpy()
    
    n_examples = x.shape[0]
    d = x.shape[1]
    
    #for updating the weigths properly
    y_aux = []
    x_aux = x_biased
    for j in range(n_examples):
        if y[j] == 0:
            y_aux.append(-1)
        else:
            y_aux.append(1)
    
    for i in range(n_examples):
        x_aux[i] = y_aux[i] * x_biased[i] #changing the sign of the values 
        
    w = w_init    
    # loop for the epoch
    for k in range(epoch):
        idx = k % n_examples
        
        prod = np.dot(x_aux[idx],w)     
        if prod <= b:# missclassified or margin violated
            #update weight
            w = w + eta * x_aux[idx]
        if (np.dot(x_aux,w)>b).all(): #if any example violate the margin or is missclasified
            print(k)
            return w
        
    return w

In [69]:
def predict(w, x):
    """
    Predicts the class labels of the data matrix x
    Inputs:    w a vector with the weight values computed by the perceptron (intercept in w[0]) [list or numpy array of size d+1]
               x a feature matrix containing an example on each row [pandas DataFrame of shape n x d]
    Output:    A vector of labels (1 or 0) [numpy array of size n]
    """
    prediction =[]
    #to manage the bias term
    x_aux = x.to_numpy()
    x_biased = np.concatenate((np.ones((x_aux.shape[0], 1)), x_aux), axis=1)
    
    y_aux= np.dot(x_biased, w)
    
    for i in y_aux:
        if i < 0:
            prediction.append(0)
        else:
            prediction.append(1)          
                          
    return prediction

In [ ]:
#computed weights 
weigths = perceptron(x_train, y_train,0 ,np.zeros(x_train.shape[1]+1), 0.1, 100000000)

#predicted y
y_pred = predict( weigths , x_train)

In [ ]:
len(y_pred)

In [ ]:
counter = 0
n_examples = x_train.shape[0]

y_aux = y_train.to_numpy()
for i in range(n_examples):
    if y_pred[i] == y_aux[i]:
        counter = counter + 1

acc_score = (counter/n_examples)*100

print(acc_score)